In [1]:
# 在这个notebook中，用于 langchain 0.3 + pydantic v2 的 申论评分任务实现。 基于此实现，完成后续 ShenlunChat的重构。
# 实现以下目标
# 1. 使用LCEL 实现申论评分任务。
#    1.1 使用 pydantic v2 的 BaseModel 设计 state 
#    1.2 使用 LCEL 完成 可配置 可重试 可设置并行数 的 任务配置。
#    1.3 完成不同情况下的 失败结果退出。
# 2. 完成不同模型的评测 （方便后续对于模型的评测）
# 3. 实现 log 的配置：
#    3.1 可存储 可打印 可以配置等级
# 完成以上的类图设计 + 拓展说明

## PART One: 在此部分，会完成 langchain 新API的尝试调用，完成基本链路的走通。

# PART TWO: 在此部分，会完成 模型评测的基本链路实现。

# PART THREE: 在此部分，会完成 类图的设计。 说明在此设计下，如何能够对新的任务进行拓展。

我们设计以下几个概念:

Task: LLM的一个任务
TaskChain: 申论中的一个大任务，由多个Task组成。

对于每一个Task，都是由LCEL实现的。而LCEL 则是由 prompt + llm + parser组成。 其中 prompt则是由 role（角色）+task（任务描述）+ input_format（输入格式）+demonstrations（示例）+output_format（输出格式）+current_input（当前输入）组成。

一个TaskChain将不同的Task串起来，为了比较好的分隔开多个task，使用BaseModel定义的state来对每一个task 和 TaskChain 进行输入与输出的定义。 对于每一个task和TaskChain内的操作,我们定义抽象类，避免直接对task进行定义。


In [3]:
import qianfan
import os
from dotenv import load_dotenv
dot_env_path = "/Users/guochaoqun/Documents/code/.env"
load_dotenv(dotenv_path=dot_env_path)

ak = os.getenv("QIANFAN_AK")
sk = os.getenv("QIANFAN_SK")

qianfan.ChatCompletion(access_key=ak, secret_key=sk).models()

{'AquilaChat-7B',
 'BLOOMZ-7B',
 'ChatGLM2-6B-32K',
 'ChatLaw',
 'EB-turbo-AppBuilder',
 'ERNIE 3.5',
 'ERNIE Speed',
 'ERNIE Speed-AppBuilder',
 'ERNIE-3.5-128K',
 'ERNIE-3.5-8K',
 'ERNIE-3.5-8K-0205',
 'ERNIE-3.5-8K-0329',
 'ERNIE-3.5-8K-0613',
 'ERNIE-3.5-8K-preview',
 'ERNIE-3.5-preview',
 'ERNIE-4.0-8K',
 'ERNIE-4.0-8K-0104',
 'ERNIE-4.0-8K-0329',
 'ERNIE-4.0-8K-0613',
 'ERNIE-4.0-8K-Latest',
 'ERNIE-4.0-8K-Preview-0518',
 'ERNIE-4.0-8K-preview',
 'ERNIE-4.0-Turbo-8K',
 'ERNIE-4.0-preview',
 'ERNIE-Bot',
 'ERNIE-Bot-4',
 'ERNIE-Bot-turbo',
 'ERNIE-Bot-turbo-AI',
 'ERNIE-Character-8K',
 'ERNIE-Character-Fiction-8K',
 'ERNIE-Function-8K',
 'ERNIE-Functions-8K',
 'ERNIE-Lite-8K',
 'ERNIE-Lite-8K-0308',
 'ERNIE-Lite-8K-0922',
 'ERNIE-Lite-AppBuilder-8K-0614',
 'ERNIE-Speed',
 'ERNIE-Speed-128K',
 'ERNIE-Speed-8K',
 'ERNIE-Tiny-8K',
 'Gemma-7B-it',
 'Llama-2-13B-Chat',
 'Llama-2-70B-Chat',
 'Llama-2-7B-Chat',
 'Meta-Llama-3-70B',
 'Meta-Llama-3-8B',
 'Mixtral-8x7B-Instruct',
 'Qianfan-